In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import configparser
import os.path
from os import path
from importlib import reload
import wandb


creds_path_ar = ["../../credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

In [ ]:
if WANDB_enable == True:
    wandb.init(project="sota-mafat-base")
    os.environ['WANDB_NOTEBOOK_NAME'] = '[SS]Alexnet_pytorch'

In [ ]:
cd {PATH_ROOT}

In [ ]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
from termcolor import colored

from src.data import feat_data, get_data, get_data_pipeline
from src.models import arch_setup, base_base_model, alex_model

# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)


random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [ ]:
config = dict()
config['num_tracks'] = 3
config['val_ratio'] = 4
config['shift_segment'] = list(np.arange(1,31))
config['get_shifts'] = True
config['get_horizontal_flip'] = False
config['get_vertical_flip'] = False

In [ ]:
train_x, train_y, val_x, val_y = get_data_pipeline.pipeline_trainval(PATH_ROOT+PATH_DATA, config)

In [ ]:
print(train_x.shape)
print(val_x.shape)

In [ ]:
train_set = arch_setup.DS(train_x,train_y)
val_set= arch_setup.DS(val_x,val_y)

train_loader=DataLoader(dataset= train_set, batch_size = 32, shuffle = True, num_workers = 2)
val_loader=DataLoader(dataset= val_set, batch_size = 32, shuffle = True, num_workers = 2)

In [ ]:
model= alex_model.alex_mdf_s_model()
# model.apply(init_weights)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

model.to(device)

if WANDB_enable == True:
    wandb.init(project="sota-mafat-base")
    os.environ['WANDB_NOTEBOOK_NAME'] = '[SS]Alexnet_pytorch'
    wandb.watch(model)
    wandb.config['data_config'] = config
    wandb.config['train_size'] = train_x.shape[0]
    wandb.config['val_size'] = val_x.shape[0]
    wandb.log(config)

In [ ]:
log = arch_setup.train_epochs(train_loader,val_loader,model,criterion,optimizer,num_epochs= 10,device=device,train_y=train_y,val_y=val_y, WANDB_enable = WANDB_enable, wandb= wandb)

In [ ]:
arch_setup.plot_loss_train_test(log,model)

In [ ]:
arch_setup.plot_ROC_local_gpu(train_loader,val_loader,model,device)

In [ ]:
from src.features import specto_feat

In [ ]:
test_path = 'MAFAT RADAR Challenge - Public Test Set V1'
test_df = get_data.load_data(test_path, PATH_ROOT + PATH_DATA)
test_df = specto_feat.data_preprocess(test_df.copy())
test_x = test_df['iq_sweep_burst']
test_x = test_x.reshape(list(test_x.shape)+[1])

In [ ]:
# Creating DataFrame with the probability prediction for each segment
submission =  pd.DataFrame()
submission['segment_id'] = test_df['segment_id']
submission['prediction'] = model(torch.from_numpy(test_x).to(device).type(torch.float32)).detach().cpu().numpy()
submission['prediction'] = submission['prediction'].astype('float')

# Save submission
submission.to_csv('submission.csv', index=False)